In [22]:
#imports
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import sklearn as sk
from sklearn.datasets import make_classification
import scikitplot as skplt
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Softmax
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import random
from sklearn import svm
from sklearn.metrics import classification_report


In [2]:
#load Data
url = 'Dataset_Subsystem_2.csv';
df = pd.read_csv(url, na_values='?').dropna();
df = df.drop_duplicates(keep='first', inplace=False).reset_index(drop=True)
df = df.sample(frac=1).reset_index(drop=True)
#df = df.sample(frac=1)
X = df[['palm_root_x','palm_root_y','palm_thumb_1_x','palm_thumb_1_y','palm_thumb_2_x','palm_thumb_2_y',
           'palm_thumb_3_x','palm_thumb_3_y','palm_index_1_x','palm_index_1_y','palm_index_2_x','palm_index_2_y',
           'palm_index_3_x','palm_index_3_y','palm_index_4_x','palm_index_4_y','palm_middle_1_x','palm_middle_1_y',
           'palm_middle_2_x','palm_middle_2_y','palm_middle_3_x','palm_middle_3_y','palm_middle_4_x','palm_middle_4_y',
           'palm_ring_1_x','palm_ring_1_y','palm_ring_2_x','palm_ring_2_y','palm_ring_3_x','palm_ring_3_y','palm_ring_4_x',
           'palm_ring_4_y','palm_pinky_1_x','palm_pinky_1_y','palm_pinky_2_x','palm_pinky_2_y','palm_pinky_3_x','palm_pinky_3_y',
           'palm_pinky_4_x','palm_pinky_4_y','dorsal_root_x','dorsal_root_y','dorsal_thumb_1_x','dorsal_thumb_1_y','dorsal_thumb_2_x',
           'dorsal_thumb_2_y','dorsal_thumb_3_x','dorsal_thumb_3_y','dorsal_index_1_x','dorsal_index_1_y','dorsal_index_2_x','dorsal_index_2_y',
           'dorsal_index_3_x','dorsal_index_3_y','dorsal_index_4_x','dorsal_index_4_y','dorsal_middle_1_x','dorsal_middle_1_y','dorsal_middle_2_x',
           'dorsal_middle_2_y','dorsal_middle_3_x','dorsal_middle_3_y','dorsal_middle_4_x','dorsal_middle_4_y','dorsal_ring_1_x','dorsal_ring_1_y',
           'dorsal_ring_2_x','dorsal_ring_2_y','dorsal_ring_3_x','dorsal_ring_3_y','dorsal_ring_4_x','dorsal_ring_4_y','dorsal_pinky_1_x','dorsal_pinky_1_y',
           'dorsal_pinky_2_x','dorsal_pinky_2_y','dorsal_pinky_3_x','dorsal_pinky_3_y','dorsal_pinky_4_x','dorsal_pinky_4_y']];

df['label'] = df.apply(lambda row: row.camera_facing_side + row.gesture, axis = 1)
y = df["label"]

#values = np.array(y)
#label_encoder = LabelEncoder()
#integer_encoded = label_encoder.fit_transform(values)
#onehot_encoder = OneHotEncoder(sparse=False)
#integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
#y_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)  


In [3]:
def getScore(Xtrain,ytrain,Xtest,ytest):
    X = tf.keras.layers.Input(shape=(Xtrain.shape[1],))
    linear = tf.keras.layers.Dense(10)(X)
    softmax = tf.math.softmax(linear)
    model = tf.keras.models.Model(inputs=Xtrain, outputs=softmax)

    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-4),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()]
             )


# perform gradient descent / train
    model.fit(x=Xtrain,
              y=ytrain_encoded,
              batch_size=X_train.shape[0],
              epochs=1
         )
    
# evaluate the test accuracy
    _, test_accuracy = model.evaluate(Xtest, ytest_encoded, verbose = 0)
    print(f"The test accuracy is {test_accuracy * 100}%")

    return model.score(Xtest,ytest_encoded)

    

In [4]:
def getScore_new(Xtrain,ytrain,Xtest,ytest):
    model = svm.SVC(kernel='linear') # Linear Kernel
    #Train the model using the training sets
    model.fit(Xtrain, ytrain)
    #ypred = model.predict(Xtest)
    #print(ypred)
    
    #Predict the response for test dataset
    return model.score(Xtest,ytest)

In [29]:
#ten fold cross validation
kf = KFold(n_splits = 10) 
kf.get_n_splits(X)
K_score = [];
for train_index, test_index in kf.split(X):
        Xtrain, Xtest = X.loc[train_index], X.loc[test_index]
        ytrain, ytest = y.loc[train_index], y.loc[test_index]
        K_score.append(getScore_new(Xtrain,ytrain,Xtest,ytest))
        #getScore_new(Xtrain,ytrain,Xtest,ytest)
        #print(K_score)
print(np.mean(K_score))
        

1.0


In [27]:
def test_unseen():
    train_x = X[:25]
    test_x = X.tail(1000)
    
    train_y = y[:25]
    test_y = y.tail(1000)
    #print(train_x[:10])
    #print(test_x[:10])
    #print(train_y[:10])
    #print(test_y[:10])
    model = svm.SVC(kernel='linear') # Linear Kernel
    #Train the model using the training sets
    model.fit(train_x, train_y)
    y_pred = model.predict(test_x)
    
    #print(y_pred)
    #print("-----------------")
    #print(test_y)
    #print('-----------------')
    print(classification_report(test_y,y_pred))

In [28]:
test_unseen()

                     precision    recall  f1-score   support

         fistdorsal       0.99      0.96      0.97       164
           fistpalm       1.00      0.97      0.98       156
         opendorsal       0.96      1.00      0.98       195
           openpalm       0.95      1.00      0.97       176
three_fingersdorsal       0.96      0.93      0.94       175
  three_fingerspalm       1.00      0.99      0.99       134

           accuracy                           0.97      1000
          macro avg       0.98      0.97      0.97      1000
       weighted avg       0.97      0.97      0.97      1000

